In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from Area2_analysis.funcs import pred_with_new_weights
from Area2_analysis.funcs import nans, fit_and_predict

# Single Lag

In [ ]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD.nwb"

dataset_5ms = NWBDataset(filename, split_heldout=False)
xy_vel = dataset_5ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_5ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_5ms.resample(5)
dataset_5ms.smooth_spk(40, name='smth_40')
bin_width = dataset_5ms.bin_width
print(bin_width)

In [ ]:
n_dims = 20 # for PCA

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')


trial_mask = passive_mask
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_5ms.data.spikes_smth_40)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_5ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

## with Neurons

In [ ]:
positive_lag = 100
negative_lag = -50
x_field = 'spikes_smth_40'
y_field ='hand_acc'

# Prepare for plotting
plot_dir = [0.0,90.0, 180.0,270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red','blue', 'green','orange']
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act-pas/"

dataset = dataset_5ms

In [ ]:
ranges = [(0,120),(-100,120)]
labels = ['early_acc_','long_acc_']
for pred_range, label in zip(ranges,labels):
    lag = positive_lag
    actR2, act_weights, act_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    pasR2, pas_weights, pas_vel_df = fit_and_predict(dataset, passive_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    actOnPas_R2, actOnPas_r, actOnPas_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, lag,x_field,y_field, act_weights)

    x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(16, 8))
    # Loop through trials, plotting true and predicted velocities on appropriate subplot
    handles = []; labels = []
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset.trial_info[dataset.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in act_vel_df[np.isin(act_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            hdl = axs[0][0].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][0].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in pas_vel_df[np.isin(pas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][1].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][1].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in actOnPas_vel_df[np.isin(actOnPas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][2].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][2].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        handles.append(hdl[0])
        labels.append(trial_dir)

    # Add labels
    axs[0][0].set_title('Active trials')
    axs[0][1].set_title('Passive trials')
    axs[0][2].set_title('Active weights on Passive trials')

    axs[1][0].set_title('R2: ' + str(int(actR2*1000)/1000))
    axs[1][1].set_title('R2: ' + str(int(pasR2*1000)/1000))
    axs[1][2].set_title('R2: ' + str(int(actOnPas_R2*1000)/1000) + ', r: ' + str(int(actOnPas_r*1000)/1000))


    axs[0][0].set_ylabel(f'True {plot_dim} {y_field}')
    axs[1][0].set_ylabel(f'Predicted {plot_dim} {y_field}')
    axs[1][0].set_xlabel('Time after move onset (ms)')
    axs[1][1].set_xlabel('Time after arm bump (ms)')
    axs[1][2].set_xlabel('Time after arm bump (ms)')
    axs[0][0].set_xlim(pred_range)
    fig.suptitle('Predicting ' + str(y_field)+ ', Positive Lag')
    fig.legend(handles=handles, labels=labels, title='target dir or\nbump dir', loc='right')
    plt.tight_layout()
    plt.savefig(figDir + monkey + '_pos_' + label + plot_dim +'.png', dpi = 'figure')
    plt.close()

    lag = negative_lag
    actR2, act_weights, act_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time',pred_range, lag, x_field, y_field)
    pasR2, pas_weights, pas_vel_df = fit_and_predict(dataset, passive_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    actOnPas_R2, actOnPas_r, actOnPas_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, lag,x_field,y_field, act_weights)

    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(16, 8))
    # Loop through trials, plotting true and predicted velocities on appropriate subplot
    handles = []; labels = []
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset.trial_info[dataset.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in act_vel_df[np.isin(act_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            hdl = axs[0][0].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][0].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in pas_vel_df[np.isin(pas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][1].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][1].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in actOnPas_vel_df[np.isin(actOnPas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][2].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][2].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        handles.append(hdl[0])
        labels.append(trial_dir)

    # Add labels
    axs[0][0].set_title('Active trials')
    axs[0][1].set_title('Passive trials')
    axs[0][2].set_title('Active weights on Passive trials')

    axs[1][0].set_title('R2: ' + str(int(actR2*1000)/1000))
    axs[1][1].set_title('R2: ' + str(int(pasR2*1000)/1000))
    axs[1][2].set_title('R2: ' + str(int(actOnPas_R2*1000)/1000) + ', r: ' + str(int(actOnPas_r*1000)/1000))


    axs[0][0].set_ylabel(f'True {plot_dim} {y_field}')
    axs[1][0].set_ylabel(f'Predicted {plot_dim} {y_field}')
    axs[1][0].set_xlabel('Time after move onset (ms)')
    axs[1][1].set_xlabel('Time after arm bump (ms)')
    axs[1][2].set_xlabel('Time after arm bump (ms)')
    axs[0][0].set_xlim(pred_range)
    fig.suptitle('Predicting ' + str(y_field)+ ', Negative Lag')
    fig.legend(handles=handles, labels=labels, title='target dir or\nbump dir', loc='right')
    plt.tight_layout()
    plt.savefig(figDir + monkey + '_neg_' + label + plot_dim +'.png', dpi = 'figure')
    plt.close()

## with PCA

In [ ]:
positive_lag = 100
negative_lag = -50
x_field = 'PCA'
y_field ='hand_acc'

# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
colors = ['red', 'blue', 'green', 'orange']
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act-pas/"
dataset = dataset_5ms

In [ ]:
ranges = [(0,120),(-100,120)]
labels = ['early_acc_','long_acc_']
for pred_range, label in zip(ranges,labels):
    lag = positive_lag
    actR2, act_weights, act_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    pasR2, pas_weights, pas_vel_df = fit_and_predict(dataset, passive_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    actOnPas_R2, actOnPas_r, actOnPas_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, lag,x_field,y_field, act_weights)

    x_axis = np.arange(pred_range[0], pred_range[1], dataset.bin_width)
    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(16, 8))
    # Loop through trials, plotting true and predicted velocities on appropriate subplot
    handles = []; labels = []
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset.trial_info[dataset.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in act_vel_df[np.isin(act_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            hdl = axs[0][0].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][0].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in pas_vel_df[np.isin(pas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][1].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][1].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in actOnPas_vel_df[np.isin(actOnPas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][2].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][2].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        handles.append(hdl[0])
        labels.append(trial_dir)

    # Add labels
    axs[0][0].set_title('Active trials')
    axs[0][1].set_title('Passive trials')
    axs[0][2].set_title('Active weights on Passive trials')

    axs[1][0].set_title('R2: ' + str(int(actR2*1000)/1000))
    axs[1][1].set_title('R2: ' + str(int(pasR2*1000)/1000))
    axs[1][2].set_title('R2: ' + str(int(actOnPas_R2*1000)/1000) + ', r: ' + str(int(actOnPas_r*1000)/1000))


    axs[0][0].set_ylabel(f'True {plot_dim} {y_field}')
    axs[1][0].set_ylabel(f'Predicted {plot_dim} {y_field}')
    axs[1][0].set_xlabel('Time after move onset (ms)')
    axs[1][1].set_xlabel('Time after arm bump (ms)')
    axs[1][2].set_xlabel('Time after arm bump (ms)')
    axs[0][0].set_xlim(pred_range)
    fig.suptitle('Predicting ' + str(y_field)+ ', Positive Lag')
    fig.legend(handles=handles, labels=labels, title='target dir or\nbump dir', loc='right')
    plt.tight_layout()
    plt.savefig(figDir + monkey + '_pos_' + label + plot_dim +'.png', dpi = 'figure')
    plt.close()

    lag = negative_lag
    actR2, act_weights, act_vel_df = fit_and_predict(dataset, active_mask, 'move_onset_time',pred_range, lag, x_field, y_field)
    pasR2, pas_weights, pas_vel_df = fit_and_predict(dataset, passive_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    actOnPas_R2, actOnPas_r, actOnPas_vel_df = pred_with_new_weights(dataset, passive_mask, 'move_onset_time',pred_range, lag,x_field,y_field, act_weights)

    fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(16, 8))
    # Loop through trials, plotting true and predicted velocities on appropriate subplot
    handles = []; labels = []
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset.trial_info[dataset.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in act_vel_df[np.isin(act_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            hdl = axs[0][0].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][0].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in pas_vel_df[np.isin(pas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][1].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][1].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        for _, trial in actOnPas_vel_df[np.isin(actOnPas_vel_df.trial_id, cond_ids)].groupby('trial_id'):
            axs[0][2].plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
            axs[1][2].plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        handles.append(hdl[0])
        labels.append(trial_dir)

    # Add labels
    axs[0][0].set_title('Active trials')
    axs[0][1].set_title('Passive trials')
    axs[0][2].set_title('Active weights on Passive trials')

    axs[1][0].set_title('R2: ' + str(int(actR2*1000)/1000))
    axs[1][1].set_title('R2: ' + str(int(pasR2*1000)/1000))
    axs[1][2].set_title('R2: ' + str(int(actOnPas_R2*1000)/1000) + ', r: ' + str(int(actOnPas_r*1000)/1000))


    axs[0][0].set_ylabel(f'True {plot_dim} {y_field}')
    axs[1][0].set_ylabel(f'Predicted {plot_dim} {y_field}')
    axs[1][0].set_xlabel('Time after move onset (ms)')
    axs[1][1].set_xlabel('Time after arm bump (ms)')
    axs[1][2].set_xlabel('Time after arm bump (ms)')
    axs[0][0].set_xlim(pred_range)
    fig.suptitle('Predicting ' + str(y_field)+ ', Negative Lag')
    fig.legend(handles=handles, labels=labels, title='target dir or\nbump dir', loc='right')
    plt.tight_layout()
    plt.savefig(figDir + monkey + '_neg_' + label + plot_dim +'.png', dpi = 'figure')
    plt.close()